# Hunyuan3D Generation Analysis

This notebook runs the Hunyuan3D Gradio app and provides access to generation statistics for analysis and visualization.

In [ ]:
# Import Required Libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import os
import numpy as np
from datetime import datetime

# Configure plotting
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)

print("Libraries imported successfully!")

## Run Gradio App and Capture Stats

**Instructions:**
1. Run the cell below to launch the Gradio app
2. Generate some 3D models using the web interface
3. The generation statistics will be automatically captured in global variables
4. Use subsequent cells to analyze the captured data

In [ ]:
# Run the Gradio app - this will start the web interface
# Change directory to the Hunyuan3D-2GP folder first
import os
os.chdir(r"c:\Users\HP\Desktop\hunyun3d\Hunyuan3D-2GP")

# Run the gradio app (this will block until you generate some models)
# Use Ctrl+C to stop the app and return to the notebook
%run gradio_app.py --mini --low-vram-mode

# Note: After running this cell, the app will start. 
# Generate some 3D models in the web interface, then stop the app to continue analysis.

## Access Generation Statistics

Now you can access the captured generation data:

In [ ]:
# Access the captured generation statistics
try:
    # Get the last generation
    last_stats = get_last_generation()
    
    # Get all generation history
    all_generations = get_generation_history()
    
    if last_stats:
        print("=== LAST GENERATION STATS ===")
        print(f"Total generations captured: {len(all_generations)}")
        print(f"Prompt: {last_stats['generation_info']['prompt']}")
        print(f"Folder: {last_stats['generation_info']['folder_name']}")
        print(f"Directory: {last_stats['generation_info']['folder_directory']}")
        
        if 'white_mesh' in last_stats['generation_info']:
            mesh_info = last_stats['generation_info']['white_mesh']
            print(f"White Mesh: {mesh_info['mesh_name']} ({mesh_info['mesh_format']})")
            
        if 'textured_mesh' in last_stats['generation_info']:
            mesh_info = last_stats['generation_info']['textured_mesh']
            print(f"Textured Mesh: {mesh_info['mesh_name']} ({mesh_info['mesh_format']})")
            
        print(f"Faces: {last_stats['number_of_faces']}")
        print(f"Vertices: {last_stats['number_of_vertices']}")
        
        # Display the complete stats structure
        print("\n=== COMPLETE STATS STRUCTURE ===")
        print(json.dumps(last_stats, indent=2, default=str))
        
    else:
        print("No generation data found. Please run the Gradio app and generate some 3D models first.")
        
except NameError:
    print("Generation functions not available. Please run the gradio_app.py first using %run.")

## Analyze Timing Performance

Parse and visualize timing data from generation statistics:

In [ ]:
# Analyze timing performance
if last_stats and 'time' in last_stats:
    timing_data = last_stats['time']
    
    # Create a DataFrame for timing analysis
    times_df = pd.DataFrame(list(timing_data.items()), columns=['Stage', 'Time (seconds)'])
    times_df = times_df[times_df['Stage'] != 'total']  # Exclude total for breakdown analysis
    
    print("=== TIMING BREAKDOWN ===")
    for stage, time_sec in timing_data.items():
        print(f"{stage.title()}: {time_sec:.2f} seconds")
    
    # Create visualization
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Bar chart of timing breakdown
    ax1.bar(times_df['Stage'], times_df['Time (seconds)'])
    ax1.set_title('Generation Time Breakdown')
    ax1.set_xlabel('Processing Stage')
    ax1.set_ylabel('Time (seconds)')
    ax1.tick_params(axis='x', rotation=45)
    
    # Pie chart of time distribution
    ax2.pie(times_df['Time (seconds)'], labels=times_df['Stage'], autopct='%1.1f%%')
    ax2.set_title('Time Distribution by Stage')
    
    plt.tight_layout()
    plt.show()
    
    # Performance metrics
    total_time = timing_data.get('total', sum(times_df['Time (seconds)']))
    print(f"\n=== PERFORMANCE METRICS ===")
    print(f"Total Generation Time: {total_time:.2f} seconds")
    print(f"Average Stage Time: {np.mean(times_df['Time (seconds)']):.2f} seconds")
    print(f"Longest Stage: {times_df.loc[times_df['Time (seconds)'].idxmax(), 'Stage']} ({times_df['Time (seconds)'].max():.2f}s)")
    print(f"Fastest Stage: {times_df.loc[times_df['Time (seconds)'].idxmin(), 'Stage']} ({times_df['Time (seconds)'].min():.2f}s)")
    
else:
    print("No timing data available. Please generate a 3D model first.")

## Examine Model Information

Access model configuration and parameters:

In [ ]:
# Examine model information
if last_stats:
    print("=== MODEL CONFIGURATION ===")
    model_info = last_stats.get('model', {})
    params_info = last_stats.get('params', {})
    
    print("Shape Generation Model:")
    print(f"  Path: {model_info.get('shapegen', 'N/A')}")
    
    print("Texture Generation Model:")
    print(f"  Path: {model_info.get('texgen', 'N/A')}")
    
    print("\n=== GENERATION PARAMETERS ===")
    for param, value in params_info.items():
        print(f"{param.replace('_', ' ').title()}: {value}")
    
    # Create a DataFrame for parameters
    params_df = pd.DataFrame(list(params_info.items()), columns=['Parameter', 'Value'])
    
    print("\n=== PARAMETERS TABLE ===")
    print(params_df.to_string(index=False))
    
    # Extract key variables for easy access
    prompt = last_stats['generation_info'].get('prompt')
    folder_name = last_stats['generation_info'].get('folder_name')
    folder_dir = last_stats['generation_info'].get('folder_directory')
    
    print(f"\n=== KEY VARIABLES FOR NOTEBOOK ACCESS ===")
    print(f"prompt = '{prompt}'")
    print(f"folder_name = '{folder_name}'")
    print(f"folder_directory = '{folder_dir}'")
    
    # Make variables globally accessible
    globals()['current_prompt'] = prompt
    globals()['current_folder_name'] = folder_name
    globals()['current_folder_directory'] = folder_dir
    globals()['current_stats'] = last_stats
    
    print("\nVariables now available globally:")
    print("- current_prompt")
    print("- current_folder_name") 
    print("- current_folder_directory")
    print("- current_stats")
    
else:
    print("No model information available. Please generate a 3D model first.")

## Extract Mesh Properties

Analyze the generated mesh statistics and file information:

In [ ]:
# Extract mesh properties
if last_stats:
    print("=== MESH STATISTICS ===")
    num_faces = last_stats.get('number_of_faces', 0)
    num_vertices = last_stats.get('number_of_vertices', 0)
    
    print(f"Number of Faces: {num_faces:,}")
    print(f"Number of Vertices: {num_vertices:,}")
    
    # Calculate mesh complexity metrics
    if num_faces > 0 and num_vertices > 0:
        face_to_vertex_ratio = num_faces / num_vertices
        print(f"Face-to-Vertex Ratio: {face_to_vertex_ratio:.2f}")
        
        # Estimate mesh complexity
        if num_faces < 1000:
            complexity = "Low"
        elif num_faces < 10000:
            complexity = "Medium"
        elif num_faces < 50000:
            complexity = "High"
        else:
            complexity = "Very High"
        print(f"Mesh Complexity: {complexity}")
    
    # Extract mesh file information
    generation_info = last_stats.get('generation_info', {})
    
    if 'white_mesh' in generation_info:
        white_mesh = generation_info['white_mesh']
        print(f"\n=== WHITE MESH INFO ===")
        print(f"Filename: {white_mesh.get('mesh_name', 'N/A')}")
        print(f"Format: {white_mesh.get('mesh_format', 'N/A')}")
        print(f"Path: {white_mesh.get('mesh_path', 'N/A')}")
        print(f"File exists: {os.path.exists(white_mesh.get('mesh_path', ''))}")
        
        # Get file size if it exists
        mesh_path = white_mesh.get('mesh_path', '')
        if os.path.exists(mesh_path):
            file_size = os.path.getsize(mesh_path)
            print(f"File size: {file_size / (1024*1024):.2f} MB")
    
    if 'textured_mesh' in generation_info:
        textured_mesh = generation_info['textured_mesh']
        print(f"\n=== TEXTURED MESH INFO ===")
        print(f"Filename: {textured_mesh.get('mesh_name', 'N/A')}")
        print(f"Format: {textured_mesh.get('mesh_format', 'N/A')}")
        print(f"Path: {textured_mesh.get('mesh_path', 'N/A')}")
        print(f"File exists: {os.path.exists(textured_mesh.get('mesh_path', ''))}")
        
        # Get file size if it exists
        mesh_path = textured_mesh.get('mesh_path', '')
        if os.path.exists(mesh_path):
            file_size = os.path.getsize(mesh_path)
            print(f"File size: {file_size / (1024*1024):.2f} MB")
    
    # Store mesh info in global variables for easy access
    globals()['mesh_faces'] = num_faces
    globals()['mesh_vertices'] = num_vertices
    globals()['mesh_files'] = generation_info
    
    print(f"\n=== MESH VARIABLES FOR NOTEBOOK ACCESS ===")
    print(f"mesh_faces = {num_faces}")
    print(f"mesh_vertices = {num_vertices}")
    print("mesh_files = generation_info dictionary")
    
else:
    print("No mesh data available. Please generate a 3D model first.")

## Visualize Statistics

Create comprehensive visualizations of the generation data:

In [ ]:
# Create comprehensive visualizations
if last_stats and len(all_generations) > 0:
    
    # If we have multiple generations, analyze trends
    if len(all_generations) > 1:
        print(f"=== GENERATION HISTORY ANALYSIS ({len(all_generations)} generations) ===")
        
        # Extract data for comparison
        gen_data = []
        for i, gen in enumerate(all_generations):
            gen_data.append({
                'Generation': i + 1,
                'Faces': gen.get('number_of_faces', 0),
                'Vertices': gen.get('number_of_vertices', 0),
                'Total_Time': gen.get('time', {}).get('total', 0),
                'Shape_Time': gen.get('time', {}).get('shape generation', 0),
                'Prompt': gen.get('generation_info', {}).get('prompt', 'N/A')[:30] + "..."
            })
        
        df = pd.DataFrame(gen_data)
        
        # Create multi-panel visualization
        fig, axes = plt.subplots(2, 2, figsize=(16, 12))
        
        # Plot 1: Mesh complexity over generations
        axes[0,0].plot(df['Generation'], df['Faces'], 'o-', label='Faces', linewidth=2, markersize=8)
        axes[0,0].plot(df['Generation'], df['Vertices'], 's-', label='Vertices', linewidth=2, markersize=8)
        axes[0,0].set_title('Mesh Complexity Over Generations')
        axes[0,0].set_xlabel('Generation Number')
        axes[0,0].set_ylabel('Count')
        axes[0,0].legend()
        axes[0,0].grid(True, alpha=0.3)
        
        # Plot 2: Generation time trends
        axes[0,1].bar(df['Generation'], df['Total_Time'], alpha=0.7, color='skyblue')
        axes[0,1].set_title('Total Generation Time')
        axes[0,1].set_xlabel('Generation Number')
        axes[0,1].set_ylabel('Time (seconds)')
        axes[0,1].grid(True, alpha=0.3)
        
        # Plot 3: Shape generation time vs total time
        axes[1,0].scatter(df['Shape_Time'], df['Total_Time'], s=100, alpha=0.7, c=df['Generation'], cmap='viridis')
        axes[1,0].set_title('Shape Time vs Total Time')
        axes[1,0].set_xlabel('Shape Generation Time (seconds)')
        axes[1,0].set_ylabel('Total Time (seconds)')
        axes[1,0].grid(True, alpha=0.3)
        
        # Plot 4: Face-to-vertex ratio
        face_vertex_ratio = df['Faces'] / df['Vertices']
        axes[1,1].bar(df['Generation'], face_vertex_ratio, alpha=0.7, color='lightcoral')
        axes[1,1].set_title('Face-to-Vertex Ratio')
        axes[1,1].set_xlabel('Generation Number')
        axes[1,1].set_ylabel('Ratio')
        axes[1,1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        # Summary statistics
        print("\n=== SUMMARY STATISTICS ===")
        print(f"Average faces: {df['Faces'].mean():.0f}")
        print(f"Average vertices: {df['Vertices'].mean():.0f}")
        print(f"Average generation time: {df['Total_Time'].mean():.2f} seconds")
        print(f"Fastest generation: {df['Total_Time'].min():.2f} seconds")
        print(f"Slowest generation: {df['Total_Time'].max():.2f} seconds")
        
    else:
        # Single generation visualization
        print("=== SINGLE GENERATION VISUALIZATION ===")
        
        # Create a summary visualization for single generation
        fig, axes = plt.subplots(2, 2, figsize=(16, 10))
        
        # Timing breakdown
        timing_data = last_stats.get('time', {})
        if timing_data:
            times = [(k, v) for k, v in timing_data.items() if k != 'total']
            stages = [x[0] for x in times]
            values = [x[1] for x in times]
            
            axes[0,0].barh(stages, values, color='lightblue')
            axes[0,0].set_title('Processing Stage Times')
            axes[0,0].set_xlabel('Time (seconds)')
        
        # Mesh properties
        faces = last_stats.get('number_of_faces', 0)
        vertices = last_stats.get('number_of_vertices', 0)
        
        mesh_data = ['Faces', 'Vertices']
        mesh_values = [faces, vertices]
        
        axes[0,1].bar(mesh_data, mesh_values, color=['orange', 'green'], alpha=0.7)
        axes[0,1].set_title('Mesh Properties')
        axes[0,1].set_ylabel('Count')
        
        # Generation parameters pie chart
        params = last_stats.get('params', {})
        numeric_params = {k: v for k, v in params.items() if isinstance(v, (int, float)) and k != 'seed'}
        
        if numeric_params:
            axes[1,0].pie(numeric_params.values(), labels=numeric_params.keys(), autopct='%1.1f%%')
            axes[1,0].set_title('Generation Parameters')
        
        # Simple text summary
        axes[1,1].text(0.1, 0.8, f"Prompt: {last_stats['generation_info']['prompt'][:50]}...", 
                      transform=axes[1,1].transAxes, fontsize=12, wrap=True)
        axes[1,1].text(0.1, 0.6, f"Folder: {last_stats['generation_info']['folder_name']}", 
                      transform=axes[1,1].transAxes, fontsize=10)
        axes[1,1].text(0.1, 0.4, f"Total Time: {timing_data.get('total', 0):.2f}s", 
                      transform=axes[1,1].transAxes, fontsize=10)
        axes[1,1].text(0.1, 0.2, f"Complexity: {faces:,} faces, {vertices:,} vertices", 
                      transform=axes[1,1].transAxes, fontsize=10)
        axes[1,1].set_title('Generation Summary')
        axes[1,1].axis('off')
        
        plt.tight_layout()
        plt.show()
    
    print("\n=== DATA ACCESS SUMMARY ===")
    print("You now have access to these variables in your notebook:")
    print("• current_prompt - The generation prompt")
    print("• current_folder_name - UUID folder name")
    print("• current_folder_directory - Full folder path")
    print("• current_stats - Complete generation statistics")
    print("• mesh_faces - Number of faces in the mesh")
    print("• mesh_vertices - Number of vertices in the mesh")
    print("• mesh_files - Mesh file information")
    print("• get_last_generation() - Function to get latest stats")
    print("• get_generation_history() - Function to get all generations")
    
else:
    print("No generation data available for visualization.")
    print("Please run the Gradio app and generate some 3D models first.")

## Example: Using Captured Variables

Here are some examples of how to use the captured variables in your own analysis:

In [ ]:
# Example usage of captured variables
try:
    print("=== EXAMPLE VARIABLE USAGE ===")
    
    # Access prompt and folder information
    print(f"Generated prompt: {current_prompt}")
    print(f"Saved in folder: {current_folder_name}")
    print(f"Full path: {current_folder_directory}")
    
    # Access mesh statistics
    print(f"Mesh complexity: {mesh_faces:,} faces, {mesh_vertices:,} vertices")
    
    # Calculate custom metrics
    if mesh_faces > 0 and mesh_vertices > 0:
        complexity_score = (mesh_faces + mesh_vertices) / 1000
        print(f"Custom complexity score: {complexity_score:.2f}")
    
    # Access timing data
    timing_data = current_stats['time']
    shape_gen_time = timing_data.get('shape generation', 0)
    total_time = timing_data.get('total', 0)
    
    if total_time > 0:
        shape_percentage = (shape_gen_time / total_time) * 100
        print(f"Shape generation took {shape_percentage:.1f}% of total time")
    
    # List generated files
    print(f"\n=== GENERATED FILES ===")
    for mesh_type, mesh_info in mesh_files.items():
        if isinstance(mesh_info, dict):
            file_path = mesh_info.get('mesh_path', '')
            if os.path.exists(file_path):
                file_size = os.path.getsize(file_path) / (1024*1024)  # MB
                print(f"{mesh_type}: {mesh_info['mesh_name']} ({file_size:.2f} MB)")
    
    # Example: Create a simple custom analysis
    print(f"\n=== CUSTOM ANALYSIS EXAMPLE ===")
    efficiency_score = mesh_faces / max(total_time, 1)  # faces per second
    print(f"Generation efficiency: {efficiency_score:.0f} faces per second")
    
    # Save analysis to file
    analysis_file = os.path.join(current_folder_directory, 'analysis_summary.txt')
    with open(analysis_file, 'w') as f:
        f.write(f"Generation Analysis Summary\n")
        f.write(f"==========================\n")
        f.write(f"Prompt: {current_prompt}\n")
        f.write(f"Generation time: {total_time:.2f} seconds\n")
        f.write(f"Mesh faces: {mesh_faces:,}\n")
        f.write(f"Mesh vertices: {mesh_vertices:,}\n")
        f.write(f"Efficiency: {efficiency_score:.0f} faces/second\n")
    
    print(f"Analysis saved to: {analysis_file}")
    
except NameError as e:
    print(f"Variable not available: {e}")
    print("Please run the previous cells to capture generation data first.")
    
print("\n=== NEXT STEPS ===")
print("You can now:")
print("1. Access any of the captured variables (current_prompt, current_folder_directory, etc.)")
print("2. Perform custom analysis using mesh_faces, mesh_vertices, timing data")
print("3. Load and analyze the generated mesh files using trimesh or other libraries")
print("4. Create custom visualizations with the captured data")
print("5. Export analysis results to files in the generation folder")